In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import transforms

In [2]:
#load and label the data
df = pd.read_csv('../data/city.csv', low_memory=False)
df['assessor_id'] = df['assessor_id'].str[1:]
df = transforms.add_labels(df)

In [3]:
df.shape

(18406, 239)

In [4]:
clean = transforms.Preprocessing()

In [5]:
df = clean.transform(df)

In [6]:
df.shape

(17351, 81)

In [7]:
df.isnull().sum().sum() #was 32,986

0

In [8]:
neg = df['labels'].value_counts()[0]

In [9]:
pos = df['labels'].value_counts()[1]

In [10]:
pos

1631

In [14]:
pos / (df.shape[0])

0.24847653869591713

In [12]:
balance = transforms.BalanceClasses()

In [13]:
df = balance.transform(df)